In [ ]:
! mkdir -p datasets
%cd datasets
! wget -nc https://raw.githubusercontent.com/pablonoya/zigzag-ml/master/datasets/housing.csv
%cd ..

# Descenso del gradiente
> Es un algoritmo **iterativo** de **optimización** para encontrar el **mínimo local** de una función.

Cuando un modelo de machine learning aprende, no va mejorando, en realidad, va siendo "menos malo" 😎.   
Para lograr esto definimos una **_loss function_** también llamada *cost function* o **función de costo** que mide el **error** de nuestro modelo y utilizamos la **gradiente**, de ahí el nombre del algoritmo, la cual es un vector de derivadas que apunta al **máximo** de dicha función con una pendiente que es más o menos inclinada según el error que se mida en ese punto 📉.

<img src="https://cdn-images-1.medium.com/max/1000/1*RxU9mwBejyPoxM95_p-gEA.gif" width=500/>

## Llegar al mínimo ¿apuntando al máximo?  
Como la idea es ser menos malo, nos vamos **alejando** del punto **máximo** en la función de costo, en este punto, el **error es muy alto**. Caminaremos sobre la función de costo **dando pasos hacia atrás** hasta alcanzar el punto **mínimo** 🚶.

Estos puntos de la función están determinados por los parámetros, $m$ y $b$ en el caso de la recta, y los datos de entrenamiento.  
Comparamos el **modelo**, dado por los datos y sus parámetros, con los **labels**, que deberían ser la respuesta ideal del modelo.

Por cada **iteración del algoritmo**, un modelo de regresión lineal cambiará más o menos de esta forma:

<img src="https://miro.medium.com/max/1280/1*eeIvlwkMNG1wSmj3FR6M2g.gif" width=400/>

## Función de costo
Esta cambia según el modelo que usemos, para las regresiones lineales se usa la diferencia entre predicción y *label* elevada al cuadrado, como ocurre con la métrica MSE 😉.

Esta función tiene la forma $y = x^2$, además es [convexa](https://es.wikipedia.org/wiki/Funci%C3%B3n_convexa).  
Lo cual significa que tendrá un sólo mínimo, al cual denominamos **mínimo global** pues no habrá otro, que denominaríamos **mínimo local** en caso de una función más compleja:

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/b6/Extrema_example_es.svg/1200px-Extrema_example_es.svg.png" width=400/>

La ubicación del **punto mínimo** de esta función **dependerá de los datos** de entrenamiento, y cambia de lugar si realizamos transformaciones o añadimos features.  
Debido a estas variaciones, de antemano no sabemos dónde está exactamente, ni por dónde empezar a buscarlo 😅.

## Iteraciones
El algoritmo del descenso por el gradiente propone los siguientes pasos :

1. **Iniciar los valores de los parámetros al azar**, lo cual nos pondrá en un punto cualquiera 😃.
2. Medir el error en ese punto y **actualizar los parámetros** para disminuir ese error ⬇️.
3. **Iterar** el paso anterior, esto es, **repetirlo varias un número de veces** que le indiquemos 🔁.
4. Muestro modelo tiene el **error mínimo**, lo que denominamos **convergencia** 😉.

El modelo ha aprendido 🧠.

Y podemos controlar qué tanto cambiarán los parámetros mediante el **learning rate** o ratio de aprendizaje.

## Learning rate
Podríamos decir que controla el tamaño de los pasos que damos hacia atrás para actualizar los parámetros, y debemos tener cuidado cuando definimos este hiperparámetro ⚠️.

Si el **learning rate** fuera **alto**, el modelo bien podría converger en **menos iteraciones**, pero si fuese **demasiado alto** daríamos pasos tan grandes que nos pasaríamos del punto mínimo, saltando la convergencia ☹.

Y si fuera **bajo**, el modelo **tardaría más en converger**, aunque dado el suficiente tiempo llegaríamos al punto mínimo.  
Lento pero seguro 🐢.

<img src="https://www.math.purdue.edu/~nwinovic/figures/learning_rates.png" width=600/>

# Implementación
Vimos bastantes conceptos, ¿verdad?, la intución es muy importante para pasar a la implementación 😄.

Este algoritmo permite que *machine learning* sea *machine learning*, pues define una serie de pasos para que la máquina aprenda de los datos 🧠.  
Para implementarlo debemos definir algunas fórmulas matemáticas, que son específicas para la regresión lineal.

## Fórmula de la función de costo
Es muy parecida a MSE, con la diferencia de que multiplicamos todo por la constante $\tfrac{1}{2m}$  

$$ J(\theta) = \frac{1}{2m} \sum_{i=1}^{m} ( h_\theta(x^{(i)}) - y^{(i)} )^2$$

Donde:
+ $m$ es la cantidad de datos.
+ $\theta$ es el vector de parámetros.
+ $i$ es el número de la fila en nuestra matriz de datos.
+ $h_\theta(x) = \theta^T X = \theta_0 + \theta_1x_1 + \cdots + \theta_nx_n$
+ $x^{(i)}$ es una fila de datos.
+ $y^{(i)}$ es la etiqueta de esa fila de datos.


La hipótesis $h_\theta(x)$ es equivalente a $\hat{y}$. Representa al modelo de regresión lineal, no incluimos $x_0$ porque esta es igual a 1, quedando sólo $\theta_0$ como el término independiente.

### Operaciones vectorizadas
Para la computadora, suele ser más eficiente tratar directamente con vectores y matrices, que implementaremos usando **arrays de numpy**, muchas formulas usan una **notación vectorizada** como $\theta^T X$ para las **sumatorias de productos** aprovechando [cómo se calculan](https://www.problemasyecuaciones.com/matrices/multiplicar-matrices-producto-matricial-ejemplos-explicados-propiedades-matriz.html) las **multiplicaciones de matrices**, cuyo requisito es que el número de **columnas** de la primera sea **igual** al número de **filas** de la segunda, es decir sus **dimensiones** deben ser **(n, k)** y **(k, m)** respectivamente. El resultado será una matriz con dimensiones **(n, m)**. 

Esto significa que las matrices a multiplicar deben **compartir un mismo número**, en nuestro caso multiplicando `theta` por `X` ¿qué numero comparten?  
`theta` es el vector de features y `X` es la matriz de datos de entrada, cada fila es un dato, y las columnas son... ¡features!, ambas **comparten el número de features** 😉

Para multiplicar usaremos el operador `@` porque así lo definió numpy, pero debemos ponerlos en el **orden correcto** la operación `theta @ X` multiplicaría arrays de `shape` **(f, 1)** y **(m, f)**, pero así no cumplimos el requisito ☹.  
La fórmula $\theta^T X$ multiplica la traspuesta de theta, la implementamos como `theta.T`, esto **voltea las dimensiones** de **(f, 1)** a **(1, f)** pero esto tampoco cumple el requsito ☹.

La solución es multiplicar **(m, f)** x **(f, 1)** dando como resultado una matriz de **(m, 1)** igual a las dimensiones de $y$, esto nos permitirá luego restarla de la hipótesis elemento por elemento.  
Fijarnos en qué haremos con el resultado es una buena idea para acomodar nuestras multiplicaciones de matrices, no importa que no sigamos la fórmula tal cual 😉
A menos que terminemos con una forma muy diferente y que no podamos utilizar, como (1, 1) 😅

In [3]:
import numpy as np

def loss(X, y, theta):
    # el orden correcto (m, f) x (f, 1)
    # resultará en (m, 1) igual que y
    h = X @ theta
    
    # sólo transcribimos la fórmula, por la suma devolverá un número
    return 1 / (2*m) * np.sum((h - y) ** 2)

## Gradiente
Este es el vector de derivadas de la función de costo:

$$\frac{1}{m} \sum_{i=1}^{m} (h_\theta(x^{(i)}) - y^{(i)})x_j^{(i)} = \frac{1}{m} (h_\theta(X) - Y) X$$

Donde $x_j^{(i)}$ es un dato en la fila número $i$ columna número $j$

Recuerda que la función de costo tiene la forma $x^2$ cuya derivada es $2x$ este número multiplicado por $\frac{1}{2m}$ cancela la constante a $\frac{1}{m}$

Por su definición, debemos devolver un vector de shape **(f, 1)** porque usamos la gradiente para actualizar `theta` usando el **learning rate**.

Como en la función de costo, la operación `h - y` tendrá shape de **(m, 1)**, luego debemos multiplicar este resultado por X que tiene shape **(m, f)**.

Para multiplicar, podemos trasponer la resta `h - y`, multiplicando **(1, m) x (m, f)**  
o `X` para multiplicar **(f, m) x (m, 1)**, tomaremos esta opción, pues resulta en **(f, 1)** como necesitamos 😉

Además eliminamos la sumatoria gracias a la multiplicación de matrices😃.

In [2]:
def grad(X, y, theta):
    h = X @ theta
    # cambiamos de lugar los términos
    return 1/m * X.T @ (h - y)

## Actualización
Por ultimo es importante utilizar lo que se conoce como la regla de actualización:

$$\theta_j := \theta_j - \alpha * grad \text{ (actualizar simultáneamente $\theta_j$ para todas las  $j$)}$$

Donde $\alpha$ es el **learning rate** y $j$ es el número de columna

Simultáneamente significa **al mismo tiempo**, esta nota es importante porque **no debemos volver a calcular el gradiente** hasta haber **actualizado antes todos los parámetros**, pues la misma gradiente depende de los parámetros 😵‍💫.

## \# Código final
La implementación es un poco diferente a las fórmulas, esto pasa por las shapes de nuestras variables, y porque resumimos la sumatoria de productos en forma la multiplicación de matrices para el caso de la gradiente 😃.  

> Por esto es **importante fijarse en las shapes**, tanto las que tenemos como las que deseamos lograr ⚠️.

Usaremos los datos de entrenamiento que estuvimos usando antes, asegurando que que `y` sea una matriz de (m, 1).  
En el caso de `X` le añadiremos una columna de unos, estos representan a $x_0$ para multiplicarlos con $\theta_0$, lo cual resulta en el término independiente.

In [7]:
import pandas as pd

data_housing = pd.read_csv('./datasets/housing.csv')
data_housing.dropna(inplace=True)

np.random.seed(42)

X = data_housing['median_income']
y = data_housing['median_house_value']

# añadimos la columna de unos
X = np.column_stack((np.ones(len(X)), X))
X

array([[1.    , 8.3252],
       [1.    , 8.3014],
       [1.    , 7.2574],
       ...,
       [1.    , 1.7   ],
       [1.    , 1.8672],
       [1.    , 2.3886]])

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test,y_train,y_test = train_test_split(X, y, test_size=0.2)

Ahora definamos algunos hiperparámetros como **alpha** o el **número máximo de iteraciones**.  
Tambien definimos `theta` que tendrá 2 elementos iniciados al azar: nuestra feature y el término independiente.

In [42]:
# alpha suele ser un valor bajito
alpha = 0.0003

# Tendremos 1000 iteraciones
max_iters = 1000

# empezamos con un vector de números aleatorios
# tendrá 2 filas y 1 columna
theta = np.random.randn(2, 1)

Entonces iniciamos con las iteraciones utilizando nuestras funciones de costo y gradiente 😉

In [43]:
# número de elementos
m = len(y_train)

# El viejo reshape
y_train = y_train.reshape(-1, 1)

for i in range(max_iters):
    # veamos cómo disminuye el costo cada 100 iteraciones
    if i % 100 == 0:
        costo = loss(X_train, y_train, theta)
        print(f"Costo en it. # {i} {costo :.2f}")
        
    # primero calculamos el gradiente
    gradiente = grad(X_train, y_train, theta)
    # actualizamos theta, dando el paso atrás con la resta
    theta -= alpha * gradiente

Costo en it. # 0 27970926073.83
Costo en it. # 100 11180501087.93
Costo en it. # 200 5954283193.15
Costo en it. # 300 4326966880.29
Costo en it. # 400 3819672773.89
Costo en it. # 500 3660949793.75
Costo en it. # 600 3610714879.02
Costo en it. # 700 3594251780.51
Costo en it. # 800 3588307983.82
Costo en it. # 900 3585646982.24


El costo ha bajado, ¡el modelo ha aprendido! 🧠

# Ejercicios
Prueba cambiando el **learning rate**, puedes ir en una escala de tres: 0.0001, 0.0003, 0.001, ...  
Mientras sea mayor, necesitarás menos **iteraciones** 😉 pero no te excedas o el modelo no aprenderá 🤯

In [44]:
# sólo es un hiperparámetro


Añade más features y ajusta los hiperparámetros, es posible que no llegues a los resultados de sklearn, pero lo importante es que tu costo siempre vaya bajando

In [45]:
# no te olvides de cambiar la forma de theta


# La importancia de (transformar) los datos
El descenso del gradiente no es lo más importante para que las máquinas aprendan, son los **datos**, porque se aprende de ellos 🧠.  
Y el cómo los **transformemos antes de entrenar un modelo** influirá en los resultados que obtengamos, como en la **regresión polinómica** obtuvimos una **curva** sólo transformando los datos.

Trabajar con los datos antes del entrenamiento se conoce como [preprocesamiento](6_preprocesamiento.ipynb).

## Feature scaling
Si tuvieramos dos features nuestra función de costo sería en 3D, nuestro objetivo de llegar al punto mínimo se mantiene y se vería más o menos así:  

<img src="https://suniljangirblog.files.wordpress.com/2018/12/1-1.gif" width=350>

Pero estas features pueden tener rangos muy **diferentes**, por ejemplo, en nuestro dataset RM tiene un rango diferente a LSTAT 

In [ ]:
from sklearn.datasets import load_boston

X, y = load_boston(True)

print(f"Rango RM    {min(X[:, 5])} -  {max(X[:, 5])}")
print(f"Rango LSTAT {min(X[:, 12])} -  {max(X[:, 12])}")

Esto causa que nuestra función de costo quede así, vista desde arriba:

![features_unscaled](img/5.1.4_features_unscaled.png)

Si iniciamos el descenso por uno de los extremos más alejados de esta elipse, nos tomará más tiempo llegar al mínimo global que tenemos en el centro, la solución es **escalar** las features para que tengamos un círculo con las features en un rango de (-1, 1) o (0, 1).

Escalar al primer rango se conoce como **standarization**, y al segundo como **normalization**. Podemos aplicar ambos con sklearn, generalmente **no** se escala la variable target, el objetivo a predecir.

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

np.random.seed(42)

X_selected = X[:, [5, 7]]
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.4)

norm = MinMaxScaler()
stand = StandardScaler()
X_train_norm = norm.fit_transform(X_train)
X_train_stand = stand.fit_transform(X_train)

print("\t Sin escalar \t|\t Normalization \t|\t Standarization")
print( np.column_stack((X_train, X_train_norm, X_train_stand))[:5])

Podemos ver que standarization tiene negativos por su rango (-1, 1) también tiene la característica de llevar las features a una media de 0, es recomendable usarla cuando nuestros datos siguen una [distribución normal](https://es.wikipedia.org/wiki/Distribuci%C3%B3n_normal), en caso contrario, es recomendable usar normalization.

Pero son recomendaciones, es más importante probar y evaluar lo que mejor se ajuste a nuestros datos 😉